In [1]:
%%html
<style>.container{width:99%}</style>

In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import os
import shutil

import requests

from serverless import settings
from serverless.pipeline.utils import download_url
from serverless.pipeline.data import census_businesspatterns, census_population
from serverless.pipeline import Hera

In [ ]:
for folder, subfolder in settings.etl.tiger.items():
    
    raw_path = settings.root_path/settings.data.raw_path/subfolder['file_path']
    interim_path = settings.root_path/settings.data.interim_path/subfolder['file_path']
    processed_path = settings.root_path/settings.data.processed_path/subfolder['file_path']
    url = subfolder.url
    
    for file in subfolder.files:
        raw_file_path = raw_path/file
        processed_file_path = processed_path/file
        interim_file_path = interim_path/file
        # raw_file_path.mkdir(parents=True, exist_ok=True)
        # processed_file_path.mkdir(parents=True, exist_ok=True)
        raw_file_exists = os.path.isfile(raw_file_path)
        interim_file_exists = os.path.isfile(interim_path)
        processed_file_exists = os.path.isfile(processed_file_path)
        
        if not raw_file_exists:
            print(url+file)
            print('DOWNLOADING')
            download_url(url+file, raw_file_path)
            if '.zip' in file and not interim_file_exists:
                print('UNZIPPING')
                shutil.unpack_archive(raw_file_path, interim_file_path.parent)

https://www2.census.gov/geo/tiger/TIGER2021/ZCTA520/tl_2021_us_zcta520.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/COUNTY/tl_2021_us_county.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_01_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_02_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_04_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_05_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_06_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_08_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_09_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_10_tract.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_11_tra

UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_31_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_32_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_33_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_34_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_35_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_36_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_37_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_38_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_39_tabblock20.zip
DOWNLOADING
UNZIPPING
https://www2.census.gov/geo/tiger/TIGER2021/TABBLOCK20/tl_2021_

In [ ]:
dst_path = Path(f'{settings.data.raw_path}/{folder}/{subfolder}/')
processed_path = Path(f'{settings.data.processed_path}/{folder}/{subfolder}/')
dst_path.mkdir(parents=True, exist_ok=True)

    for file in files:
        unquoted_filename = requests.utils.unquote(file)
        filename = dst_path/unquoted_filename
        file_exists = os.path.isfile(filename)
        processed_file_exists = Path(unquoted_filename).stem in (processed_path/unquoted_filename).parts[-1]

        if not file_exists:
            download_url(url+file, filename)
            if '.zip' in file:
                shutil.unpack_archive(filename, dst_path)

        if not processed_file_exists:
            if partition_function:
                if folder == 'school' and subfolder == 'zone':
                    school_zone_files = [i for i in os.listdir(dst_path) if i.endswith('.shp')]
                    for school_zone_file in school_zone_files:
                        school_zone_file_exists = os.path.isfile(dst_path/school_zone_file.replace('.shp', '.geojson'))
                        if not school_zone_file_exists:
                            print(dst_path, school_zone_file)
                            partition_function(dst_path/school_zone_file, partitions)
                else:
                    print(filename)
                    geo_filename = str(filename).replace('.zip', '.shp')
                    partition_function(geo_filename, partitions)